# Model Metrics

In [186]:
import pandas as pd
import os
from glob import glob
from azureml.core import Workspace, Run
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go

In [2]:
ws = Workspace.from_config()

## Load model run log

In [50]:
RUN_LOG = 'training_scripts/runids.csv'

In [51]:
df = pd.read_csv(RUN_LOG)
df.columns

Index(['runid', 'model_name', 'data_subset', 'start_time'], dtype='object')

In [52]:
df.head()

,runid,model_name,data_subset,start_time
0,birdsongs_2_1618101588_9cdc9314,cnn1_audin_nmel_1,kaggle_10sec_wav,"04/10/2021, 20:39:52"
1,birdsongs_2_1618101601_6c54df25,cnn1_audin_nmel_2,kaggle_10sec_wav,"04/10/2021, 20:40:03"
2,birdsongs_2_1618101679_621085b4,cnn1_audin_nmel_dcbl_1,kaggle_10sec_wav,"04/10/2021, 20:41:23"
3,birdsongs_2_1618101691_08bb6e9d,cnn1_audin_nmel_dcbl_2,kaggle_10sec_wav,"04/10/2021, 20:41:34"
4,birdsongs_2_1618105893_94d00361,cnn1_audin_nffthl_1,kaggle_10sec_wav,"04/10/2021, 21:51:37"


In [53]:
# drop test runs
df = df[df['runid'].str.split('_').str[0] != 'test']
df.head()

,runid,model_name,data_subset,start_time
0,birdsongs_2_1618101588_9cdc9314,cnn1_audin_nmel_1,kaggle_10sec_wav,"04/10/2021, 20:39:52"
1,birdsongs_2_1618101601_6c54df25,cnn1_audin_nmel_2,kaggle_10sec_wav,"04/10/2021, 20:40:03"
2,birdsongs_2_1618101679_621085b4,cnn1_audin_nmel_dcbl_1,kaggle_10sec_wav,"04/10/2021, 20:41:23"
3,birdsongs_2_1618101691_08bb6e9d,cnn1_audin_nmel_dcbl_2,kaggle_10sec_wav,"04/10/2021, 20:41:34"
4,birdsongs_2_1618105893_94d00361,cnn1_audin_nffthl_1,kaggle_10sec_wav,"04/10/2021, 21:51:37"


In [54]:
# get keep the last run of each model/data_subset pair
df_lastruns = df.drop_duplicates(subset=['model_name', 'data_subset'], 
                                 keep='last').copy()
df_lastruns.head()

,runid,model_name,data_subset,start_time
0,birdsongs_2_1618101588_9cdc9314,cnn1_audin_nmel_1,kaggle_10sec_wav,"04/10/2021, 20:39:52"
1,birdsongs_2_1618101601_6c54df25,cnn1_audin_nmel_2,kaggle_10sec_wav,"04/10/2021, 20:40:03"
2,birdsongs_2_1618101679_621085b4,cnn1_audin_nmel_dcbl_1,kaggle_10sec_wav,"04/10/2021, 20:41:23"
3,birdsongs_2_1618101691_08bb6e9d,cnn1_audin_nmel_dcbl_2,kaggle_10sec_wav,"04/10/2021, 20:41:34"
4,birdsongs_2_1618105893_94d00361,cnn1_audin_nffthl_1,kaggle_10sec_wav,"04/10/2021, 21:51:37"


## Get from Azure

In [55]:
ws = Workspace.from_config()

### Get run context

In [56]:
# get run context from Azure for each runid
runs = {}
for runid in tqdm(df_lastruns['runid']):
    run = Run.get(ws, runid)
    runs[runid] = run

100%|██████████| 33/33 [00:13<00:00,  2.43it/s]


In [57]:
# eliminate failed runs
status = []
for runid, run in runs.items():
    status.append(run.status)
    
df_lastruns['status'] = status
df_lastruns.head()

,runid,model_name,data_subset,start_time,status
0,birdsongs_2_1618101588_9cdc9314,cnn1_audin_nmel_1,kaggle_10sec_wav,"04/10/2021, 20:39:52",Completed
1,birdsongs_2_1618101601_6c54df25,cnn1_audin_nmel_2,kaggle_10sec_wav,"04/10/2021, 20:40:03",Completed
2,birdsongs_2_1618101679_621085b4,cnn1_audin_nmel_dcbl_1,kaggle_10sec_wav,"04/10/2021, 20:41:23",Completed
3,birdsongs_2_1618101691_08bb6e9d,cnn1_audin_nmel_dcbl_2,kaggle_10sec_wav,"04/10/2021, 20:41:34",Completed
4,birdsongs_2_1618105893_94d00361,cnn1_audin_nffthl_1,kaggle_10sec_wav,"04/10/2021, 21:51:37",Completed


In [58]:
# get only completed runs
df_complete = df_lastruns[df_lastruns['status'] == 'Completed']
df_complete = df_complete.set_index('runid')
df_complete.head()

,model_name,data_subset,start_time,status
runid,,,,
birdsongs_2_1618101588_9cdc9314,cnn1_audin_nmel_1,kaggle_10sec_wav,"04/10/2021, 20:39:52",Completed
birdsongs_2_1618101601_6c54df25,cnn1_audin_nmel_2,kaggle_10sec_wav,"04/10/2021, 20:40:03",Completed
birdsongs_2_1618101679_621085b4,cnn1_audin_nmel_dcbl_1,kaggle_10sec_wav,"04/10/2021, 20:41:23",Completed
birdsongs_2_1618101691_08bb6e9d,cnn1_audin_nmel_dcbl_2,kaggle_10sec_wav,"04/10/2021, 20:41:34",Completed
birdsongs_2_1618105893_94d00361,cnn1_audin_nffthl_1,kaggle_10sec_wav,"04/10/2021, 21:51:37",Completed


### Download run files

In [59]:
# download files for each run

OUT_DIR = 'model_data'
os.makedirs(OUT_DIR, exist_ok=True)
for idx, row in df_complete.iterrows():
    
    # check if the files already exist
    file_path = os.path.join(OUT_DIR, f'{row.model_name}-{idx}*.h5')
    files = glob(file_path)
    if len(files):
        print(f'files for {idx} are already downloaded.')
    else:
        print(f'downloading {idx} files')
        runs[idx].download_files(prefix='outputs', 
                                       output_directory=OUT_DIR, 
                                       append_prefix=False)

files for birdsongs_2_1618101588_9cdc9314 are already downloaded.
files for birdsongs_2_1618101601_6c54df25 are already downloaded.
files for birdsongs_2_1618101679_621085b4 are already downloaded.
files for birdsongs_2_1618101691_08bb6e9d are already downloaded.
files for birdsongs_2_1618105893_94d00361 are already downloaded.
files for birdsongs_2_1618105905_9510b3e7 are already downloaded.
files for birdsongs_2_1618109592_7b50fea5 are already downloaded.
files for birdsongs_2_1618109605_d3804748 are already downloaded.
files for birdsongs_2_1618147392_4d00cd64 are already downloaded.
files for birdsongs_2_1618147406_e8ed4602 are already downloaded.
files for birdsongs_2_1618150013_c2f1f4bf are already downloaded.
files for birdsongs_2_1618153363_7017415d are already downloaded.
files for birdsongs_2_1618153718_0f7f4e95 are already downloaded.
files for birdsongs_2_1618153737_76aab195 are already downloaded.
files for birdsongs_2_1618157490_39253a0e are already downloaded.
files for 

## Model History

### Training History

In [82]:
# get run metrics from save file
run_metrics = []
for idx, row in df_complete.iterrows():    
    file = os.path.join(OUT_DIR, f'{row.model_name}-{idx}' + '.history')
    with open(file, 'rb') as f:
        metric = pickle.load(f)
        metric['runid'] = idx
        metric['data_subset'] = row['data_subset']
    run_metrics.append(metric)
run_metrics = pd.DataFrame(run_metrics)
run_metrics.head(2)

,loss,categorical_accuracy,top_3_accuracy,top_5_accuracy,val_loss,val_categorical_accuracy,val_top_3_accuracy,val_top_5_accuracy,lr,runid,data_subset
0,"[5.553964614868164, 5.017866134643555, 4.44454...","[0.008900728076696396, 0.03586278483271599, 0....","[0.027676714584231377, 0.08666840195655823, 0....","[0.04320426285266876, 0.12649428844451904, 0.2...","[5.344659805297852, 4.781640529632568, 4.24124...","[0.018168604001402855, 0.05741279199719429, 0....","[0.039244186133146286, 0.13372093439102173, 0....","[0.0625, 0.19040697813034058, 0.31213662028312...","[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.0...",birdsongs_2_1618101588_9cdc9314,kaggle_10sec_wav
1,"[5.668341159820557, 5.208738803863525, 4.79221...","[0.007276507094502449, 0.02507796324789524, 0....","[0.02176455222070217, 0.06477390974760056, 0.1...","[0.03436850383877754, 0.0953742191195488, 0.17...","[5.413963317871094, 5.012693405151367, 4.79702...","[0.014171511866152287, 0.032340116798877716, 0...","[0.03815406933426857, 0.08430232852697372, 0.1...","[0.053415697067976, 0.13299418985843658, 0.181...","[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.0...",birdsongs_2_1618101601_6c54df25,kaggle_10sec_wav


### Test metrics

In [83]:
# get test metrics from saved test metrics
test_metrics = []
for idx, row in df_complete.iterrows():    
    file = os.path.join(OUT_DIR, f'{row.model_name}-{idx}-test_metrics.plk')
    with open(file, 'rb') as f:
        metric = pickle.load(f)
    metric['runid'] = idx
    metric['model_name'] = row.model_name
    test_metrics.append(metric)
test_metrics = pd.DataFrame(test_metrics)
test_metrics.head(3)

,test_categorical_accuracy,test_top_3_accuracy,test_top_5_accuracy,runid,model_name
0,0.324375,0.493437,0.572500,birdsongs_2_1618101588_9cdc9314,cnn1_audin_nmel_1
1,0.283125,0.454062,0.540625,birdsongs_2_1618101601_6c54df25,cnn1_audin_nmel_2
2,0.304688,0.480000,0.573750,birdsongs_2_1618101679_621085b4,cnn1_audin_nmel_dcbl_1


### Join into a single DataFrame
This DataFrame will be used here for visualizations and in the project dashboard.

In [84]:
df_metrics = run_metrics.set_index('runid').join(test_metrics.set_index('runid'))
df_metrics.head(2)

,loss,categorical_accuracy,top_3_accuracy,top_5_accuracy,val_loss,val_categorical_accuracy,val_top_3_accuracy,val_top_5_accuracy,lr,data_subset,test_categorical_accuracy,test_top_3_accuracy,test_top_5_accuracy,model_name
runid,,,,,,,,,,,,,,
birdsongs_2_1618101588_9cdc9314,"[5.553964614868164, 5.017866134643555, 4.44454...","[0.008900728076696396, 0.03586278483271599, 0....","[0.027676714584231377, 0.08666840195655823, 0....","[0.04320426285266876, 0.12649428844451904, 0.2...","[5.344659805297852, 4.781640529632568, 4.24124...","[0.018168604001402855, 0.05741279199719429, 0....","[0.039244186133146286, 0.13372093439102173, 0....","[0.0625, 0.19040697813034058, 0.31213662028312...","[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.0...",kaggle_10sec_wav,0.324375,0.493437,0.572500,cnn1_audin_nmel_1
birdsongs_2_1618101601_6c54df25,"[5.668341159820557, 5.208738803863525, 4.79221...","[0.007276507094502449, 0.02507796324789524, 0....","[0.02176455222070217, 0.06477390974760056, 0.1...","[0.03436850383877754, 0.0953742191195488, 0.17...","[5.413963317871094, 5.012693405151367, 4.79702...","[0.014171511866152287, 0.032340116798877716, 0...","[0.03815406933426857, 0.08430232852697372, 0.1...","[0.053415697067976, 0.13299418985843658, 0.181...","[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.0...",kaggle_10sec_wav,0.283125,0.454062,0.540625,cnn1_audin_nmel_2


In [85]:
# save this file for use in the dashboard
df_metrics.to_pickle('df_metrics.pkl')

## Plot model test performance

In [134]:
# combine model name and Data Subset
df_plot = df_metrics.copy()
df_plot['model'] = df_plot['model_name'] + '<br>' + df_plot['data_subset']
df_plot['model_num'] = range(1, len(df_plot) + 1)
df_plot['model_num_text'] = 'Model #' + df_plot['model_num'].astype('str')
df_plot.head(3)

,loss,categorical_accuracy,top_3_accuracy,top_5_accuracy,val_loss,val_categorical_accuracy,val_top_3_accuracy,val_top_5_accuracy,lr,data_subset,test_categorical_accuracy,test_top_3_accuracy,test_top_5_accuracy,model_name,model,model_num,model_num_text
runid,,,,,,,,,,,,,,,,,
birdsongs_2_1618101588_9cdc9314,"[5.553964614868164, 5.017866134643555, 4.44454...","[0.008900728076696396, 0.03586278483271599, 0....","[0.027676714584231377, 0.08666840195655823, 0....","[0.04320426285266876, 0.12649428844451904, 0.2...","[5.344659805297852, 4.781640529632568, 4.24124...","[0.018168604001402855, 0.05741279199719429, 0....","[0.039244186133146286, 0.13372093439102173, 0....","[0.0625, 0.19040697813034058, 0.31213662028312...","[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.0...",kaggle_10sec_wav,0.324375,0.493437,0.572500,cnn1_audin_nmel_1,cnn1_audin_nmel_1<br>kaggle_10sec_wav,1,Model #1
birdsongs_2_1618101601_6c54df25,"[5.668341159820557, 5.208738803863525, 4.79221...","[0.007276507094502449, 0.02507796324789524, 0....","[0.02176455222070217, 0.06477390974760056, 0.1...","[0.03436850383877754, 0.0953742191195488, 0.17...","[5.413963317871094, 5.012693405151367, 4.79702...","[0.014171511866152287, 0.032340116798877716, 0...","[0.03815406933426857, 0.08430232852697372, 0.1...","[0.053415697067976, 0.13299418985843658, 0.181...","[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.0...",kaggle_10sec_wav,0.283125,0.454062,0.540625,cnn1_audin_nmel_2,cnn1_audin_nmel_2<br>kaggle_10sec_wav,2,Model #2
birdsongs_2_1618101679_621085b4,"[5.546006202697754, 5.219290256500244, 4.70500...","[0.00513253640383482, 0.01396829541772604, 0.0...","[0.015007796697318554, 0.038916319608688354, 0...","[0.0257276501506567, 0.0641242191195488, 0.169...","[5.376589775085449, 4.9785356521606445, 4.3890...","[0.014171511866152287, 0.024709302932024002, 0...","[0.030159883201122284, 0.06758721172809601, 0....","[0.04832848906517029, 0.10901162773370743, 0.2...","[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.0...",kaggle_10sec_wav,0.304688,0.480000,0.573750,cnn1_audin_nmel_dcbl_1,cnn1_audin_nmel_dcbl_1<br>kaggle_10sec_wav,3,Model #3


In [135]:
# create DataFrame with only columns we need for plot
df_plot = df_plot[['model_name',
                   'data_subset',
                   'model_num_text',
                   'test_categorical_accuracy', 
                   'test_top_3_accuracy', 
                   'test_top_5_accuracy']]
df_plot.head(3)

,model_name,data_subset,model_num_text,test_categorical_accuracy,test_top_3_accuracy,test_top_5_accuracy
runid,,,,,,
birdsongs_2_1618101588_9cdc9314,cnn1_audin_nmel_1,kaggle_10sec_wav,Model #1,0.324375,0.493437,0.572500
birdsongs_2_1618101601_6c54df25,cnn1_audin_nmel_2,kaggle_10sec_wav,Model #2,0.283125,0.454062,0.540625
birdsongs_2_1618101679_621085b4,cnn1_audin_nmel_dcbl_1,kaggle_10sec_wav,Model #3,0.304688,0.480000,0.573750


In [137]:
# rename columns for plot
df_plot.columns = ['Model Name',
                   'Data',
                   'Model No.',
                   'Categorical Accuracy',
                   'Top 3 Categorical Accuracy',
                   'Top 5 Categorical Accuracy']

# flatten dataframe for plot
df_plot = df_plot.melt(id_vars=['Model Name', 'Data', 'Model No.'],
                       var_name='Accuracy Metric',
                       value_vars=['Categorical Accuracy', 
                                      'Top 3 Categorical Accuracy', 
                                      'Top 5 Categorical Accuracy'],
                       value_name='Accuracy')
df_plot.head()

,Model Name,Data,Model No.,Accuracy Metric,Accuracy
0,cnn1_audin_nmel_1,kaggle_10sec_wav,Model #1,Categorical Accuracy,0.324375
1,cnn1_audin_nmel_2,kaggle_10sec_wav,Model #2,Categorical Accuracy,0.283125
2,cnn1_audin_nmel_dcbl_1,kaggle_10sec_wav,Model #3,Categorical Accuracy,0.304688
3,cnn1_audin_nmel_dcbl_2,kaggle_10sec_wav,Model #4,Categorical Accuracy,0.268750
4,cnn1_audin_nffthl_1,kaggle_10sec_wav,Model #5,Categorical Accuracy,0.302500


In [152]:
# accuracy plot with plotly for dashboard
fig = px.bar(df_plot.sort_values('Accuracy'), x='Model No.', 
             y='Accuracy', color='Accuracy Metric', 
             barmode='group',
             hover_name='Model No.',
             hover_data=['Model Name', 'Data'],
             title='Test Accuracy by model name')

fig.update_layout(
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    annotations = [
        dict(xref='paper',
             yref='paper',
             x=0, y=1.13,
             showarrow=False,
             text ='Hover to see Model Name and Dataset'
        )
    ]
)


## Rebuild best model

In [153]:
# Import model generator function
from training_scripts.remote.models import MODELS

In [154]:
# Get best model
best_model = df_metrics.sort_values('test_categorical_accuracy').iloc[-1]
best_model

loss                         [5.497860908508301, 4.885035991668701, 4.38679...
categorical_accuracy         [0.011783854104578495, 0.03430989757180214, 0....
top_3_accuracy               [0.03079427033662796, 0.08678385615348816, 0.1...
top_5_accuracy               [0.04804687574505806, 0.13105468451976776, 0.2...
val_loss                     [5.5985918045043945, 4.945390701293945, 4.7319...
val_categorical_accuracy     [0.01308139506727457, 0.046148255467414856, 0....
val_top_3_accuracy           [0.037427324801683426, 0.10792151093482971, 0....
val_top_5_accuracy           [0.055959302932024, 0.1587935984134674, 0.2601...
lr                           [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.0...
data_subset                                         kaggle_full_length_npy_aug
test_categorical_accuracy                                             0.600625
test_top_3_accuracy                                                   0.740625
test_top_5_accuracy                                 

In [155]:
fname = f'model_data/{best_model.model_name}-{best_model.name}.h5'

In [156]:
model = MODELS[df_metrics.loc[idx].model_name]()

In [157]:
model.load_weights(fname)

In [158]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
stft_1 (STFT)                (None, 427, 1025, 1)      0         
_________________________________________________________________
magnitude_1 (Magnitude)      (None, 427, 1025, 1)      0         
_________________________________________________________________
apply_filterbank_1 (ApplyFil (None, 427, 256, 1)       0         
_________________________________________________________________
magnitude_to_decibel_1 (Magn (None, 427, 256, 1)       0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 427, 256, 1)       4         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 427, 256, 16)      160       
_________________________________________________________________
batch_normalization_16 (Batc (None, 427, 256, 16)     

In [325]:
colors = ['#B75B37','#8BC3C7','#503653']

fig = px.bar(df_tmp, x='Accuracy Metric', 
             y='Accuracy', 
             barmode='group', color_discrete_sequence=['#EFCC91'],
             width=700, height=400,
             title='Best Model Accuracy')

fig.update_layout(
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="left",
        x=-0.7),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

### Best Model Accuracy and Val Accuracy

Fix, this works for best model, but in the dashboard it has to work when multiple models are selected.

The answer probably still to explode two dataframes but keeping both the mode name and the dataset.
The do not need to be concatenated because the traces are added to the model separatly.

Wait, this is done in a loop so it can work on a single Series.


In [181]:
# Create DataFrame for Plotting with plotly
df_tmp = pd.DataFrame(best_model['categorical_accuracy'], 
                      columns=['categorical_accuracy'])
df_tmp.index = pd.Series(range(1, len(df_tmp) + 1), name='Epochs')
df_tmp['model'] = best_model['model_name'] + '/' + best_model['data_subset']
df_tmp.head(3)

,categorical_accuracy,model
Epochs,,
1,0.011784,milsed_7block_dense/kaggle_full_length_npy_aug
2,0.034310,milsed_7block_dense/kaggle_full_length_npy_aug
3,0.081315,milsed_7block_dense/kaggle_full_length_npy_aug


In [169]:
best_model

loss                         [5.497860908508301, 4.885035991668701, 4.38679...
categorical_accuracy         [0.011783854104578495, 0.03430989757180214, 0....
top_3_accuracy               [0.03079427033662796, 0.08678385615348816, 0.1...
top_5_accuracy               [0.04804687574505806, 0.13105468451976776, 0.2...
val_loss                     [5.5985918045043945, 4.945390701293945, 4.7319...
val_categorical_accuracy     [0.01308139506727457, 0.046148255467414856, 0....
val_top_3_accuracy           [0.037427324801683426, 0.10792151093482971, 0....
val_top_5_accuracy           [0.055959302932024, 0.1587935984134674, 0.2601...
lr                           [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.0...
data_subset                                         kaggle_full_length_npy_aug
test_categorical_accuracy                                             0.600625
test_top_3_accuracy                                                   0.740625
test_top_5_accuracy                                 

In [180]:
# Create DataFrame for Plotting with plotly
df_tmp_val = pd.DataFrame(best_model['val_categorical_accuracy'], 
                      columns=['val_categorical_accuracy'])
df_tmp_val.index = pd.Series(range(1, len(df_tmp_val) + 1), name='Epochs')

df_tmp_val.head(3)

,val_categorical_accuracy,model
Epochs,,
1,0.013081,milsed_7block_dense/kaggle_full_length_npy_aug
2,0.046148,milsed_7block_dense/kaggle_full_length_npy_aug
3,0.079215,milsed_7block_dense/kaggle_full_length_npy_aug


In [194]:
df_plt = pd.concat([df_tmp, df_tmp_val['val_categorical_accuracy']], axis=1)
df_plt['Epochs'] = range(1, len(df_plt)+1)
df_plt['model_name'] = best_model['model_name'] + '<br>' + best_model['data_subset']

df_plt.head()

,categorical_accuracy,model,val_categorical_accuracy,Epochs,model_name
Epochs,,,,,
1,0.011784,milsed_7block_dense/kaggle_full_length_npy_aug,0.013081,1,milsed_7block_dense<br>kaggle_full_length_npy_aug
2,0.034310,milsed_7block_dense/kaggle_full_length_npy_aug,0.046148,2,milsed_7block_dense<br>kaggle_full_length_npy_aug
3,0.081315,milsed_7block_dense/kaggle_full_length_npy_aug,0.079215,3,milsed_7block_dense<br>kaggle_full_length_npy_aug
4,0.124870,milsed_7block_dense/kaggle_full_length_npy_aug,0.119186,4,milsed_7block_dense<br>kaggle_full_length_npy_aug
5,0.169271,milsed_7block_dense/kaggle_full_length_npy_aug,0.175145,5,milsed_7block_dense<br>kaggle_full_length_npy_aug


In [199]:
color = 'blue'
fig = go.Figure()
#fig.add_trace(go.Scatter(y=df_plt['categorical_accuracy'], x=df_plt['Epochs']))


fig.add_trace(go.Scatter(y=df_plt['categorical_accuracy'], 
                         x=df_plt['Epochs'],
                         name='<b>Accuracy:</b><br>' + df_plt['model_name'].iloc[0],
                         line = dict(color=color, dash='solid')))
fig.add_trace(go.Scatter(y=df_plt['val_categorical_accuracy'], 
                         x=df_plt['Epochs'],
                         name='<b>Val. Accccuracy:</b><br>' + df_plt['model_name'].iloc[0],
                         line = dict(color=color, dash='dash')))

fig.update_layout(go.Layout(title="Accuracy vs Validation Accuracy"))
